<a href="https://colab.research.google.com/github/VinoJose/Tensorflow/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning with TensoFlow Part 1: Feature extraction

In [2]:
# Are we running a GPU
!nvidia-smi

Sat Dec 11 13:31:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading the data

In [3]:
# Get data (10% of the food classes from Food101)
import zipfile

# Download the data
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2021-12-11 13:31:39--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   190MB/s    in 0.8s    

2021-12-11 13:31:40 (190 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
# How many images in each folder?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} and {len(filenames)} in the folder {dirpath}.")

There are 2 and 0 in the folder 10_food_classes_10_percent.
There are 10 and 0 in the folder 10_food_classes_10_percent/test.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/sushi.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/pizza.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/hamburger.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/chicken_wings.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/ice_cream.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/steak.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/ramen.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/fried_rice.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/chicken_curry.
There are 0 and 250 in the folder 10_food_classes_10_percent/test/grilled_salmon.
There are 10 and 0 in the folder 10_food_classes_10_percent/train.
There are 0 and 75 in the folder 10_foo

## Creating data loaders

In [5]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE, class_mode="categorical")

print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE, class_mode="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks (things to run whilst our model is running)

Callbacks are extra functionalities you can add to your models to be performed during or after training. Some examples are below:
* Tracking experiments with TensorBoard callback
* Model checkpoint with ModelCheckpoint callback
* Stopping a training (before it trains too long and overfits) with the EarlyStopping callback

In [6]:
# Create tensorboard callback
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datatime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to {log_dir}")
  return tensorflow_callback